In [ ]:
import socceraction.spadl as spadl
import socceraction.xthreat as xthreat
import soccerdata as sd
from pathlib import PosixPath
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
choosen_match_id = 1557659

In [ ]:
ws = sd.WhoScored(
    leagues="GER-Bundesliga2",
    seasons=[21],
    no_cache=False,
    no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    path_to_browser="/usr/bin/chromium",
    headless=True,
)

loader = ws.read_events(match_id=choosen_match_id, output_fmt="loader")

In [ ]:
df_teams = loader.teams(game_id=choosen_match_id)
df_teams

In [ ]:
df_events = loader.events(game_id=choosen_match_id)

In [ ]:
actions = {}
actions[df_events["game_id"][0]] = spadl.opta.convert_to_actions(
    df_events, df_teams["team_id"][0]
)
# 2. Convert direction of play
actions_ltr = pd.concat(
    [spadl.play_left_to_right(actions[df_events["game_id"][0]], df_teams["team_id"][0])]
)

shoot_actions = actions_ltr[actions_ltr["type_id"] == 11]
goals = shoot_actions[shoot_actions["result_id"] == 1]
goals = goals.merge(df_teams)
# shoot_actions = actions_ltr[actions_ltr['type_id'] == 11]
own_goals = actions_ltr[actions_ltr["result_id"] == 3]
own_goals = own_goals.merge(df_teams)
all_goals = pd.concat([goals, own_goals], ignore_index=True)
all_goals

In [ ]:
for team in df_teams["team_id"]:
    print(
        f"{df_teams[df_teams['team_id'] == team]['team_name'].values[0]}: {goals[goals['team_id'] == team].shape[0] + own_goals[own_goals['team_id'] != team].shape[0]}"
    )

In [ ]:
df_players = loader.players(game_id=choosen_match_id)
whole_game = max(df_players["minutes_played"])
player_dict = {}
for player_id in df_players["player_id"]:
    player_dict[player_id] = {}
    player_dict[player_id]["gI"] = 0
    player_dict[player_id]["minutes"] = df_players[
        df_players["player_id"] == player_id
    ]["minutes_played"].values[0]
    for goal_id in all_goals["original_event_id"]:
        if df_players[df_players["player_id"] == player_id]["is_starter"].values[0]:
            if df_players[df_players["player_id"] == player_id][
                "minutes_played"
            ].values[0] >= (
                (
                    (
                        all_goals[all_goals["original_event_id"] == goal_id][
                            "period_id"
                        ].values[0]
                        - 1
                    )
                    * 45
                )
                + (
                    all_goals[all_goals["original_event_id"] == goal_id][
                        "time_seconds"
                    ].values[0]
                    / 60
                )
            ):
                if (
                    all_goals[all_goals["original_event_id"] == goal_id][
                        "type_id"
                    ].values[0]
                    == 11
                ):
                    player_dict[player_id]["gI"] += (
                        1
                        if all_goals[all_goals["original_event_id"] == goal_id][
                            "team_id"
                        ].values[0]
                        == df_players[df_players["player_id"] == player_id][
                            "team_id"
                        ].values[0]
                        else -1
                    )
                else:
                    player_dict[player_id]["gI"] += (
                        -1
                        if all_goals[all_goals["original_event_id"] == goal_id][
                            "team_id"
                        ].values[0]
                        == df_players[df_players["player_id"] == player_id][
                            "team_id"
                        ].values[0]
                        else 1
                    )
        else:
            if (
                (
                    (
                        all_goals[all_goals["original_event_id"] == goal_id][
                            "period_id"
                        ].values[0]
                        - 1
                    )
                    * 45
                )
                + (
                    all_goals[all_goals["original_event_id"] == goal_id][
                        "time_seconds"
                    ].values[0]
                    / 60
                )
            ) >= (
                whole_game
                - df_players[df_players["player_id"] == player_id][
                    "minutes_played"
                ].values[0]
            ):
                if (
                    all_goals[all_goals["original_event_id"] == goal_id][
                        "type_id"
                    ].values[0]
                    == 11
                ):
                    player_dict[player_id]["gI"] += (
                        1
                        if all_goals[all_goals["original_event_id"] == goal_id][
                            "team_id"
                        ].values[0]
                        == df_players[df_players["player_id"] == player_id][
                            "team_id"
                        ].values[0]
                        else -1
                    )
                else:
                    player_dict[player_id]["gI"] += (
                        -1
                        if all_goals[all_goals["original_event_id"] == goal_id][
                            "team_id"
                        ].values[0]
                        == df_players[df_players["player_id"] == player_id][
                            "team_id"
                        ].values[0]
                        else 1
                    )

In [ ]:
gi_df = pd.DataFrame.from_dict(
    {
        "player_id": player_dict.keys(),
        "gI": [player_dict[x]["gI"] for x in player_dict.keys()],
        "minutes": [player_dict[x]["minutes"] for x in player_dict.keys()],
    }
).merge(df_players[["player_id", "player_name"]])
gi_df